In [1]:
import pandas as pd
import numpy as np

In [2]:
### England Seasons 2018/19, 2019/20, 2020/21
# for Spain and Italy I have done the same

df_en_2018_matches = pd.read_csv('data\df_en_2018_matches.csv')
df_en_2018_mins = pd.read_csv('data\df_en_2018_mins.csv')

df_en_2019_matches = pd.read_csv('data\df_en_2019_matches.csv')
df_en_2019_mins = pd.read_csv('data\df_en_2019_mins.csv')

df_en_2020_matches = pd.read_csv('data\df_en_2020_matches.csv')
df_en_2020_mins = pd.read_csv('data\df_en_2020_mins.csv')

In [3]:
# add name_team_home to df_mins
def add_name_team_home(df_matches, df_mins):
    df_new = df_matches[['id_match', 'name_team_home']]
    df_mins = pd.merge(df_mins, df_new, on = 'id_match', how = 'left')
    return(df_mins)

In [4]:
# add goal home_away team
def goal_team_difference(df):
    for i in range(0, len(df)):
        if ((df.loc[i, 'mins'] >= 0) & (df.loc[i, 'mins']<76) & (df.loc[i, 'name_team_home'] == df.loc[i, 'teamName'])):
            df.loc[i, '0-75_dif'] = 1
        elif ((df.loc[i, 'mins']>= 0) & (df.loc[i, 'mins']<76) & (df.loc[i, 'name_team_home'] != df.loc[i, 'teamName'])):
            df.loc[i, '0-75_dif'] = -1
        else:
            df.loc[i, '0-75_dif'] = 0
            
        if ((df.loc[i, 'mins'] >= 76) & (df.loc[i, 'mins']<=90) & (df.loc[i, 'name_team_home'] == df.loc[i, 'teamName'])):
            df.loc[i, '76-90+_dif'] = 1
        elif ((df.loc[i, 'mins']>= 76) & (df.loc[i, 'mins']<=90) & (df.loc[i, 'name_team_home'] != df.loc[i, 'teamName'])):
            df.loc[i, '76-90+_dif'] = -1
        else:
            df.loc[i, '76-90+_dif'] = 0            
            
    return(df)

In [5]:
# add results for periods of time
def add_goal_mins(df_work):
    # the first time
    df_work['0-75'] = df_work['mins'].apply(lambda x: 1 if (x < 75) else 0)
    df_work['75+'] = df_work['mins'].apply(lambda x: 1 if (x>= 75) else 0)
    df_work['0-15'] = df_work['mins'].apply(lambda x: 1 if ((x>= 0)  & (x<16)) else 0)
    df_work['16-30'] = df_work['mins'].apply(lambda x: 1 if ((x>= 16)  & (x<31)) else 0)
    df_work['31-45'] = df_work['mins'].apply(lambda x: 1 if ((x>= 31)  & (x<46)) else 0)

    # the second time
    df_work['46-59'] = df_work['mins'].apply(lambda x: 1 if ((x>= 46)  & (x<60)) else 0)
    df_work['60-74'] = df_work['mins'].apply(lambda x: 1 if ((x>= 60)  & (x<75)) else 0)
    df_work['75-79'] = df_work['mins'].apply(lambda x: 1 if ((x>= 75)  & (x<80)) else 0)
    df_work['80-84'] = df_work['mins'].apply(lambda x: 1 if ((x>= 80)  & (x<85)) else 0)
    df_work['85-89'] = df_work['mins'].apply(lambda x: 1 if ((x>= 85) & (x<90)) else 0)
    df_work['90+'] = df_work['mins'].apply(lambda x: 1 if (x>= 90) else 0)
    return(df_work)

In [6]:
def create_final_df_season(df_mathes, df_mins):
    df_new_mins = add_name_team_home(df_mathes, df_mins)
    df_new_mins = goal_team_difference(df_new_mins)
    df_new_mins = add_goal_mins(df_new_mins)
    
    df_id = df_new_mins.groupby('id_match').sum().reset_index()
    
    df = pd.merge(df_mathes, df_id, on = 'id_match', how = 'left')
    df = df.fillna(0)
    df = df.drop(['mins'], axis=1)
    return(df)

In [11]:
df_en_2018 = create_final_df_season(df_en_2018_matches, df_en_2018_mins)
df_en_2019 = create_final_df_season(df_en_2019_matches, df_en_2019_mins)
df_en_2020 = create_final_df_season(df_en_2020_matches, df_en_2020_mins)

df_en_2018['year'] = 2018
df_en_2019['year'] = 2019
df_en_2020['year'] = 2020

In [12]:
df_en_2020.head()

,id_match,id_team_home,name_team_home,id_team_away,name_team_away,score_HT,score_FT,0-75_dif,76-90+_dif,0-75,...,0-15,16-30,31-45,46-59,60-74,75-79,80-84,85-89,90+,year
0,592143.0,36.0,Fulham,42.0,Arsenal,0-1,0-3,-3.0,0.0,3.0,...,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2020
1,592142.0,52.0,Crystal Palace,41.0,Southampton,1-0,1-0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020
2,592144.0,40.0,Liverpool,63.0,Leeds,3-2,4-3,0.0,1.0,6.0,...,2.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,2020
3,592148.0,48.0,West Ham,34.0,Newcastle,0-0,0-2,-1.0,-1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2020
4,592147.0,60.0,West Brom,46.0,Leicester,0-0,0-3,-2.0,-1.0,2.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2020
